<a href="https://colab.research.google.com/github/jekwa1/Project/blob/main/GroupProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

In [1]:
#mount to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#load cases_malaysia csv file using Pandas
folder = '/content/drive/My Drive/HIAProject302/'
filename_1 = folder+'cases_malaysia.csv'
case_my= pd.read_csv(filename_1)

#load deaths_malaysia csv file using Pandas
folder = '/content/drive/My Drive/HIAProject302/'
filename_2 = folder+'deaths_malaysia.csv'
death_my= pd.read_csv(filename_2)

#load icu csv file using Pandas
folder = '/content/drive/My Drive/HIAProject302/'
filename_3 = folder+'icu.csv'
icu_my= pd.read_csv(filename_3)
